# Export-Third-Parties

## Token genaration and api management

In [ ]:
#@title Import modules
import requests, base64
import re
import json as js
import pandas as pd
from io import StringIO
import numpy as np
import csv
import copy

In [ ]:
#@title Token generation
#@markdown This code calls the token end point with client-id and secret

def login():
    cfg = pd.read_csv("/content/config.csv")
    client_id = cfg['client_id'].values[0]
    client_secret = cfg['client_secret'].values[0]
    token_endpoint = cfg['token_url'].values[0]
    
    data = {'grant_type': 'client_credentials'}
    
    userpass = client_id + ':' + client_secret
    encoded_u = base64.b64encode(userpass.encode()).decode()
    auth_header = "Basic " + encoded_u
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': auth_header
    }

    r = requests.post(token_endpoint, headers=headers, data=data)
    response = r.json()

    if (r.status_code == 200):
        token = response['access_token']
        # Put token in the session
        print(token)
    return token

In [ ]:
#@title Get Results 
#@markdown This code calls the end point to retrieve data
def get_results(token, request):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.get(base_url + request, headers=headers)
        if 200 <= result.status_code < 300:
          return result.text
        else:
          print(result.text)
          return  
    except:
        return

In [ ]:
#@title Post results  
#@markdown This code allows to POST data in payload or as a stream
def post_results(token, request, payload, files):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.post(base_url + request, headers=headers, data=payload, files = files)
        if 200 <= result.status_code < 300:
          json_data = js.loads(result.text)
          return json_data
        else:
          print(result.text)
          return
    except:
        return

In [ ]:
#@title Launch report
#@markdown Code to manage report launch and retrieval of data in one function
#@markdown It launchs the task and waits until completion
def run_report(token, report):
    try:
        result = post_results(token, '/v1/process-templates/'+ report +'/run',"","")
        taskId = result[0]['taskId']
        while True:
            result = get_results(token, '/v1/process-templates/'+ taskId +'/status')
            if result == "Pending" or result == "In progress":
                print ('.', end='')
            else:
                print ('\n' + result)
            if result == "Warning" or result == "Complete" or result == "Error" or result == "Cancelled":
                break
                time.sleep(1)
        print('\n')
        result = get_results(token, '/v1/process-templates/'+ report + '/files?taskId=' + taskId )
        #print(result)
        return result
    except:
        return

In [ ]:
#@title Remove empty values in json

def remove_empty_elements(d):
    """recursively remove empty lists, empty dicts, or None elements from a dictionary"""

    def empty(x):
        return x is None or x == {} or x == []

    if not isinstance(d, (dict, list)):
        return d
    elif isinstance(d, list):
        return [v for v in (remove_empty_elements(v) for v in d) if not empty(v)]
    else:
        return {k: v for k, v in ((k, remove_empty_elements(v)) for k, v in d.items()) if not empty(v)}

## Connection

In [ ]:
#@title <font color='lime'>Connect</font>
print('token:')
token = login()

## Exporting third-parties

In [ ]:
#@title <font color='lime'>Launch report on third-parties through Process Template</font>

res = run_report(token,'SU11')
buffer = StringIO(res)
df = pd.read_csv (buffer,dtype=object, delimiter = ";", usecols=[1,2,3,5,6,8,10,17,30,32,36,37,38,61,69,70,72,73,78], names = ["Data type","Code","Name","Creditor","Debtor","Other identifier type","Non resident","Country code","Account Country code","Account Name","Account Currency","Account ID type","Account ID","Account used by","Limit currency","Transaction entry limit","Account nature","Transaction max. number","Additional correspondent.One-time correspondent"],header=None, skiprows=[0])
pd.set_option('display.max_colwidth', None)
#@markdown #### Choose content type from dropdown:
type = 'json' #@param ['csv', 'json']
showEmptyValuesInJson = True#@param{type: 'boolean'}
df2 = copy.deepcopy(df)
if type == 'csv':
  df.replace(np.nan, '-', inplace = True)
  display(df)
if type == 'json':
  result = df2.to_json(orient='records')
  parsed = js.loads(result, parse_int=int)
  if not showEmptyValuesInJson:
    parsed = remove_empty_elements(parsed)
  print(js.dumps(parsed, indent=2, ensure_ascii=True) )



In [ ]:
#@title <font color='lime'>Launch report on third-parties API</font>
#@markdown #### Choose content type from dropdown:
type = 'json' #@param ['csv', 'json']
showEmptyValuesInJson = True#@param{type: 'boolean'}
res = get_results(token, '/v1/third-parties')
ppjson = js.loads(res, parse_int=int) 
if type == 'json':
  if not showEmptyValuesInJson:
    ppjson = remove_empty_elements(ppjson)
  print(js.dumps(ppjson['results'], indent=2) )
if type == 'csv': 
  df = pd.json_normalize(ppjson['results'])
  display(df)

